# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd 
import re
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
engine.table_names()

['Message']

In [5]:
df = pd.read_sql_table('Message', con=engine)
df.head()

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.shape

(26216, 41)

In [7]:
X = df.message
Y = df.loc[:, 'related':'direct_report']

In [8]:
print(X.shape)
X.head()

(26216,)


0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [9]:
print(Y.shape)
Y.head()

(26216, 36)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    #Case Normalization & remove punctuation 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) 
    
    #tokenization methods 
    
    words = word_tokenize(text)
    
    words = [w for w in words if w not in stopwords.words("english")]
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]

    
    return words

In [11]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['look', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroy', 'hospital', 'st', 'croix', 'function', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
cls = MultiOutputClassifier(DecisionTreeClassifier())

pipeline = Pipeline([
     ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', cls)
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((19662,), (6554,), (19662, 36), (6554, 36))

In [14]:
pipeline.fit(X_train.as_matrix(), y_train)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

In [15]:
y_pred = pipeline.predict(X_test)

In [27]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
for i, col in enumerate(Y.columns):
    print(f'-----------------------{i, col}----------------------------------')
    
#     y_actual = list(y_test.values[:, i])
#     y_pred = list(y_pred[:, i])
    print(classification_report(list(y_test.values[:, i]), list(y_pred[:, i])))

-----------------------(0, 'related')----------------------------------
             precision    recall  f1-score   support

          0       0.53      0.50      0.51      1565
          1       0.85      0.85      0.85      4944
          2       0.18      0.36      0.24        45

avg / total       0.77      0.76      0.76      6554

-----------------------(1, 'request')----------------------------------
             precision    recall  f1-score   support

          0       0.90      0.92      0.91      5408
          1       0.58      0.53      0.55      1146

avg / total       0.85      0.85      0.85      6554

-----------------------(2, 'offer')----------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.04      0.04      0.04        24

avg / total       0.99      0.99      0.99      6554

-----------------------(3, 'aid_related')----------------------------------
            

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f30247bcd08>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, presort=False, random_state=None,
               splitter='best'),
              n_jobs=1))],
 

In [22]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters)
cv.fit(X_train.as_matrix(), y_train)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__min_samples_leaf': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
y_pred = cv.predict(X_test)

In [24]:
for i, col in enumerate(Y.columns):
    print(f'-----------------------{i, col}----------------------------------')
    
#     y_actual = list(y_test.values[:, i])
#     y_pred = list(y_pred[:, i])
    print(classification_report(list(y_test.values[:, i]), list(y_pred[:, i])))

-----------------------(0, 'related')----------------------------------
             precision    recall  f1-score   support

          0       0.55      0.53      0.54      1565
          1       0.85      0.87      0.86      4944
          2       0.40      0.09      0.15        45

avg / total       0.78      0.78      0.78      6554

-----------------------(1, 'request')----------------------------------
             precision    recall  f1-score   support

          0       0.90      0.96      0.93      5408
          1       0.71      0.50      0.59      1146

avg / total       0.87      0.88      0.87      6554

-----------------------(2, 'offer')----------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6554

-----------------------(3, 'aid_related')----------------------------------
            

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6480
          1       0.25      0.01      0.03        74

avg / total       0.98      0.99      0.98      6554

-----------------------(27, 'other_infrastructure')----------------------------------
             precision    recall  f1-score   support

          0       0.96      0.99      0.98      6285
          1       0.37      0.08      0.13       269

avg / total       0.94      0.96      0.94      6554

-----------------------(28, 'weather_related')----------------------------------
             precision    recall  f1-score   support

          0       0.90      0.94      0.92      4742
          1       0.81      0.72      0.76      1812

avg / total       0.87      0.88      0.87      6554

-----------------------(29, 'floods')----------------------------------
             precision    recall  f1-score   support

          0       0.97      0.99      0.98      6021
         

In [25]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
moc = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = pipeline.predict(X_test)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [29]:
for i, col in enumerate(Y.columns):
    print(f'-----------------------{i, col}----------------------------------')
    
#     y_actual = list(y_test.values[:, i])
#     y_pred = list(y_pred[:, i])
    print(classification_report(list(y_test.values[:, i]), list(y_pred[:, i])))

-----------------------(0, 'related')----------------------------------
             precision    recall  f1-score   support

          0       0.64      0.48      0.55      1543
          1       0.85      0.91      0.88      4960
          2       0.27      0.33      0.30        51

avg / total       0.79      0.81      0.80      6554

-----------------------(1, 'request')----------------------------------
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5476
          1       0.79      0.44      0.56      1078

avg / total       0.88      0.89      0.88      6554

-----------------------(2, 'offer')----------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

-----------------------(3, 'aid_related')----------------------------------
            

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6273
          1       0.75      0.01      0.02       281

avg / total       0.95      0.96      0.94      6554

-----------------------(28, 'weather_related')----------------------------------
             precision    recall  f1-score   support

          0       0.86      0.96      0.91      4702
          1       0.84      0.60      0.70      1852

avg / total       0.85      0.86      0.85      6554

-----------------------(29, 'floods')----------------------------------
             precision    recall  f1-score   support

          0       0.95      0.99      0.97      5990
          1       0.87      0.39      0.54       564

avg / total       0.94      0.94      0.93      6554

-----------------------(30, 'storm')----------------------------------
             precision    recall  f1-score   support

          0       0.95      0.98      0.96      5931
          1       0.75  

### 9. Export your model as a pickle file

In [30]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

# References :

- https://stackoverflow.com/questions/29044940/how-can-you-use-a-variable-name-inside-a-python-format-specifier
- https://stackoverflow.com/questions/51911160/valueerror-mix-type-of-y-not-allowed
- https://stackoverflow.com/questions/60164560/attributeerror-series-object-has-no-attribute-as-matrix-why-is-it-error